In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key. You only have to do this once
gmaps.configure(api_key=g_key)

In [8]:
# Store the CSV saved/created in part two into a DataFrame.
vacation_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_data_df.head()

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
0,13,13,Albany,US,42.60,-73.97,82.99,65,100,9.40,moderate rain,2.91,0,NaN
1,25,25,Konongo,GH,6.62,-1.22,71.42,95,100,1.79,light rain,0.23,0,Alice Tee Hotel
2,42,42,Merauke,ID,-8.47,140.33,77.72,87,46,15.99,light rain,0.25,0,NaN
3,61,61,Lengshuijiang,CN,27.69,111.43,76.41,93,100,2.64,light rain,0.48,0,Sheraton
4,107,107,Butaritari,KI,3.07,172.79,83.10,75,100,11.39,moderate rain,2.43,0,Isles Sunset Lodge


In [21]:
# Create separte dataframe for Albany
# Albany_data_df = vacation_data_df[vacation_data_df.City_ID == 13]
Albany_data_df = vacation_data_df.loc[(vacation_data_df["City"] == 'Albany')]
Albany_data_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
0,13,13,Albany,US,42.6,-73.97,82.99,65,100,9.4,moderate rain,2.91,0,NaN


In [19]:
# Create separte dataframe for Westport
# Westport_data_df = vacation_data_df[vacation_data_df.City == 'Westport']
Westport_data_df = vacation_data_df.loc[(vacation_data_df["City"] == 'Westport')]
Westport_data_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
19,272,272,Westport,US,41.14,-73.36,73.99,94,90,20.8,light rain,0.38,0,The Westport Inn


In [22]:
# Create separte dataframe for Goderich
# Goderich_data_df = vacation_data_df[vacation_data_df.City == 'Goderich']
Goderich_data_df = vacation_data_df.loc[(vacation_data_df["City"] == 'Goderich')]
Goderich_data_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
11,199,199,Goderich,CA,43.75,-81.72,80.01,82,100,0.56,moderate rain,1.5,0,Harmony Inn


In [17]:
# Create separte dataframe for Harrison
#Harrison_data_df = vacation_data_df [vacation_data_df.City == 'Harrison']
Harrison_data_df = vacation_data_df.loc[(vacation_data_df["City"] == 'Harrison')]
Harrison_data_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
21,290,290,Harrison,US,30.41,-89.04,86.0,88,90,3.36,heavy intensity rain,4.32,0,Residence Inn by Marriott Gulfport-Biloxi Airport
